In [ ]:
import sys
import localSettings as ls
print(ls.main_path)

In [ ]:
main_path = ls.main_path
sys.path.append(main_path)

In [ ]:
from datetime import datetime
now = datetime.now()
date_time = now.strftime("%m%d%Y")
print("date and time:",date_time)

In [ ]:
SHRCALIBFACTOR = 1.00

In [ ]:
# Doing Run3?
ISRUN3 = False

In [ ]:
import plotter
import importlib
importlib.reload(plotter)
import uproot
import matplotlib.pylab as pylab
import numpy as np
import math
from sklearn.model_selection import train_test_split
import pickle
import xgboost as xgb
import nue_booster 
importlib.reload(nue_booster)
import awkward
import pandas as pd

params = {
    'axes.labelsize': 'x-large',
    'axes.titlesize': 'x-large',
    'xtick.labelsize': 'x-large',
    'ytick.labelsize': 'x-large'
}
pylab.rcParams.update(params)

In [ ]:
variables = [
    "shr_dedx_Y", "shr_bkt_pdg", "p", "pt", "selected", "nu_pdg", "shr_theta",
    "slpdg", "trk_score_v", "backtracked_pdg", # modified from shr_score_v
    "shr_pfp_id_v", "category", "shr_tkfit_dedx_Y",
    "shr_tkfit_gap10_dedx_U","shr_tkfit_gap10_dedx_V","shr_tkfit_gap10_dedx_Y",
    "shr_tkfit_2cm_dedx_U","shr_tkfit_2cm_dedx_V","shr_tkfit_2cm_dedx_Y",
    #"shr_energy_tot", 
    "trk_energy_tot", "shr_hits_tot", "ccnc", "trk_chipr",
    "trk_bkt_pdg", "hits_ratio", "n_tracks_contained", 
    #"crtveto","crthitpe","_closestNuCosmicDist",
    "NeutrinoEnergy2",
    #"run","sub","evt",
    "CosmicIP",
    "flash_pe",
    "nu_flashmatch_score","best_cosmic_flashmatch_score","best_obviouscosmic_flashmatch_score",
    #"trk_pfp_id",
    "shrmoliereavg","shrmoliererms",
    "shr_tkfit_npointsvalid","shr_tkfit_npoints", # fitted vs. all hits for shower
    "shrclusfrac0","shrclusfrac1","shrclusfrac2", # track-fitted hits / all hits
    "trkshrhitdist2", # "trkshrhitdist0","trkshrhitdist1", distance between track and shower in 2D
    "shrsubclusters0","shrsubclusters1","shrsubclusters2", # number of sub-clusters in shower
    "trk_llr_pid_score_v", # trk-PID score
    "trk_energy_proton_v", # track energy under proton hyp
    "trk_calo_energy_y_v", # track calo energy
    #"pi0_energy2_Y", # pi0 tagger variables
    "filter_pi0",
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nproton", "nu_e", "n_showers_contained", "shr_distance", "trk_distance",
    "hits_y", "shr_pz", "shr_energy", "shr_dedx_U", "shr_dedx_V", "shr_phi", "trk_phi", "trk_theta",
    "shr_tkfit_dedx_U", "shr_tkfit_dedx_V", "run", "sub", "evt", "nproton", "trk_pid_chipr_v",
    "trk_len", "mc_pdg", "slnunhits", "slnhits", "shr_score", "trk_score", "trk_hits_tot",
    "true_e_visible", "matched_E", "shr_bkt_E", "trk_bkt_E", "trk_energy", "tksh_distance", "tksh_angle",
    "npi0","npion","pion_e","muon_e","pi0truth_elec_etot",
    "pi0_e", "shr_energy_tot_cali", "shr_dedx_Y_cali", "evnunhits", "nslice", "interaction",
    "slclustfrac", "reco_nu_vtx_x", "reco_nu_vtx_y", "reco_nu_vtx_z","contained_fraction",
    "secondshower_Y_nhit","secondshower_Y_vtxdist","secondshower_Y_dot","secondshower_Y_dir","shrclusdir2",
    "shr_tkfit_2cm_nhits_Y","shr_tkfit_2cm_nhits_U","shr_tkfit_2cm_nhits_V",
    "shr_tkfit_gap10_nhits_Y","shr_tkfit_gap10_nhits_U","shr_tkfit_gap10_nhits_V",
    "weightSpline","weightTune","weightSplineTimesTune",
    # pi0 selection variables
    "pi0_radlen1","pi0_radlen2","pi0_dot1","pi0_dot2","pi0_energy1_Y","pi0_energy2_Y",
    "pi0_dedx1_fit_Y","pi0_dedx2_fit_Y","pi0_shrscore1","pi0_shrscore2","pi0_gammadot",
    "pi0_dedx1_fit_V","pi0_dedx2_fit_V","pi0_dedx1_fit_U","pi0_dedx2_fit_U",
    "pi0_mass_Y","pi0_mass_V","pi0_mass_U"
]

In [ ]:
variables = [
    "reco_nu_vtx_sce_x","reco_nu_vtx_sce_y","reco_nu_vtx_sce_z",
    "nslice","run","sub","evt","nu_e","weightSplineTimesTune"
]

In [ ]:
# nue preselection
PRESQ = 'nslice == 1'
PRESQ += ' and selected == 1'
PRESQ += ' and n_tracks_contained > 0'
PRESQ += ' and shr_energy_tot_cali > 0.07'

In [ ]:
# loose box cuts
LCUTQ = PRESQ
LCUTQ += ' and n_showers_contained == 1'
LCUTQ += ' and hits_ratio > 0.5'
LCUTQ += ' and tksh_distance < 6.0'
LCUTQ += ' and shr_tkfit_2cm_dedx_avg < 4.0'
LCUTQ += ' and tksh_angle > -0.9'
LCUTQ += ' and trkpid < 0.1'
LCUTQ += ' and shr_score < 0.40'
LCUTQ += ' and CosmicIP > 20.'

In [ ]:
# tight box cuts
TCUTQ = LCUTQ
TCUTQ += ' and tksh_distance < 4.0'
#TCUTQ += ' and (shr_tkfit_2cm_nhits_Y > 0)'
#TCUTQ += ' and (shr_tkfit_2cm_dedx_Y < 4.0 and shr_tkfit_2cm_dedx_U < 4.0 and shr_tkfit_2cm_dedx_V < 4.0)'
#TCUTQ += ' and (shr_tkfit_gap10_dedx_Y < 4.5)'
#
TCUTQ += ' and (shr_tkfit_2cm_nhits_tot > 1)'
TCUTQ += ' and (shr_tkfit_2cm_dedx_avg < 3.8)'
TCUTQ += ' and (shr_tkfit_gap10_nhits_tot > 1)'
TCUTQ += ' and (shr_tkfit_gap10_dedx_avg < 3.8)'
#
TCUTQ += ' and tksh_angle > -0.9 and tksh_angle < 0.75'
TCUTQ += ' and shrmoliereavg > 2 and shrmoliereavg < 10'
TCUTQ += ' and trkpid < 0.02'
TCUTQ += ' and n_showers_contained == 1'
TCUTQ += ' and shr_score < 0.275'
TCUTQ += ' and hits_ratio > 0.60'
TCUTQ += ' and (secondshower_Y_nhit<=8 or secondshower_Y_dot<=0.8 or anglediff_Y<=40 or secondshower_Y_vtxdist>=100)'
TCUTQ += ' and subcluster > 7'
TCUTQ += ' and trkfit < 0.70'
TCUTQ += ' and trkshrhitdist2 < 1.5'

In [ ]:
# pi0 selection
LOOSE = True
if (LOOSE):
    SCORECUT = 0.8 # 0.75 #75 # max track score
    DVTX = 3.0 # 3. # distance from vertex of each shower
    VTXDOT = 0.8 # dot product between each shower's direction and the vtx -> shr start vector
    EMIN1 =  50. # leading photon min energy
    EMIN2 =  20. #20. # 20. # subleading photon min energy
    GAMMADOT = 0.94 # max dot product between showres
    DEDXCUT = 0.0 # MeV/cm cut on leading shower only
else:
    SCORECUT = 0.5 # 0.75 #75 # max track score
    DVTX = 3.0 # 3. # distance from vertex of each shower
    VTXDOT = 0.8 # dot product between each shower's direction and the vtx -> shr start vector
    EMIN1 =  60. # leading photon min energy
    EMIN2 =  40. #20. # 20. # subleading photon min energy
    GAMMADOT = 0.94 # max dot product between showres
    DEDXCUT = 1.0 # MeV/cm cut on leading shower only

PI0SEL = 'nslice == 1'
PI0SEL += ' & pi0_shrscore1 < %f & pi0_shrscore2 < %f'%(SCORECUT,SCORECUT)
PI0SEL += '& pi0_dot1  > %f & pi0_dot2 > %f '%(VTXDOT,VTXDOT)
PI0SEL += ' & pi0_radlen1 > %f & pi0_radlen2 > %f & pi0_gammadot < %f '%(DVTX,DVTX,GAMMADOT)
PI0SEL += ' & pi0_energy1_Y > %f & pi0_energy2_Y > %f'%(EMIN1,EMIN2)
#PI0SEL += ' and (filter_pi0 == 1 or bnbdata==1 or extdata==1)'
PI0SEL += ' and pi0_dedx1_fit_Y >= %f'%DEDXCUT

In [ ]:
fold = "nuselection"
tree = "NeutrinoSelectionFilter"

'''
#NUECV = "prodgenie_bnb_nu_uboone_overlay_mcc9.1_det_var_v08_00_00_26_CV_run1_reco2_reco2.root"
NUECV = "/home/david/data/searchingfornues/v08_00_00_33/cc0pinp/0218/run1/prodgenie_bnb_intrinsice_nue_uboone_overlay_mcc9.1_v08_00_00_26_run1_reco2_reco2.root"
NUELY = "prodgenie_bnb_intrinsic_nue_uboone_overlay_mcc9_reco2_LYdown_v08_00_00_32_run1_reco2_reco2.root"
NUEWA = "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodangle_mcc9.1_v08_00_00_30_run1_reco2_reco2.root"
NUEWX = "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodx_mcc9.1_v08_00_00_30_run1_reco2_reco2.root"
NUEWYZ = "prodgenie_bnb_intrinsic_nue_uboone_overlay_wiremodyz_mcc9.1_v08_00_00_31_run1_reco2_reco2.root"
    
DETVAR_N_V = ['LY','WireMod angle','WireMod X','WireMod YZ']

DETVAR_S_V = [NUELY,NUEWA,NUEWX,NUEWYZ]
'''

#'''
NUMUCV = "prodgenie_bnb_nu_uboone_overlay_mcc9.1_det_var_v08_00_00_26_CV_run1_reco2_reco2.root"
#NUMUCV = "/home/david/data/searchingfornues/v08_00_00_33/cc0pinp/0218/run1/prodgenie_bnb_nu_uboone_overlay_mcc9.1_v08_00_00_26_filter_run1_reco2_reco2.root"
NUMULY = "prodgenie_bnb_nu_uboone_overlay_mcc9_reco2_LYdown_v08_00_00_32_run1_reco2_reco2.root"
NUMURY = "prodgenie_bnb_nu_uboone_overlay_mcc9_reco2_LYRayleigh_v08_00_00_32_run1_reco2_reco2.root"
NUMUSCE = "prodgenie_bnb_nu_uboone_overlay_mcc9_SCE_v08_00_00_34_run1_reco2_reco2.root"
NUMUR1 = "prodgenie_bnb_nu_uboone_overlay_mcc9_recombination_v08_00_00_33_recomb1_reco2_reco2.root"
NUMUR2 = "prodgenie_bnb_nu_uboone_overlay_mcc9_recombination_v08_00_00_33_recomb2_reco2_reco2.root"
NUMUR3 = "prodgenie_bnb_nu_uboone_overlay_mcc9_recombination_v08_00_00_33_recomb3_reco2_reco2.root"
NUMUWA = "prodgenie_bnb_nu_uboone_overlay_wiremodangle_mcc9.1_v08_00_00_30_run1_reco2_reco2.root"
NUMUWX = "prodgenie_bnb_nu_uboone_overlay_wiremodx_mcc9.1_v08_00_00_30_run1_reco2_reco2.root"
NUMUWYZ = "prodgenie_bnb_nu_uboone_overlay_wiremodyz_mcc9.1_v08_00_00_31_run1_reco2_reco2.root"
    
DETVAR_N_V = ['LY','Rayleigh','SCE','Recomb 1','Recomb 2','Recomb 3','WireMod angle',\
             'WireMod X','WireMod YZ']

DETVAR_S_V = [NUMULY,NUMURY,NUMUSCE,NUMUR1,NUMUR2,NUMUR3,NUMUWA,NUMUWX,NUMUWYZ]
#'''

#CV = uproot.open(NUECV)[fold][tree]
CV = uproot.open(ls.ntuple_path+NUMUCV)[fold][tree]

CVDF  = CV.pandas.df(variables, flatten=False)

CVDF['identifier'] = CVDF['run']*100000 + CVDF['evt']

CVDF.loc[ CVDF['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
CVDF.loc[ CVDF['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
CVDF.loc[ CVDF['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
CVDF.loc[ np.isnan(CVDF['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.

#'''
trk_llr_pid_v = CV.array('trk_llr_pid_score_v')
trk_energy_proton_v = CV.array('trk_energy_proton_v')
trk_calo_energy_y_v = CV.array('trk_calo_energy_y_v')
trk_id = CV.array('trk_id')-1 # I think we need this -1 to get the right result
trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
CVDF['trkpid'] = trk_llr_pid_v_sel
CVDF['protonenergy'] = trk_energy_proton_sel
CVDF['trackcaloenergy'] = trk_calo_energy_y_sel
CVDF['subcluster'] = CVDF['shrsubclusters0'] + CVDF['shrsubclusters1'] + CVDF['shrsubclusters2']
CVDF['trkfit'] = CVDF['shr_tkfit_npointsvalid'] / CVDF['shr_tkfit_npoints']
CVDF['anglediff_Y'] = np.abs(CVDF['secondshower_Y_dir']-CVDF['shrclusdir2'])

CVDF['shr_tkfit_2cm_nhits_tot'] = (CVDF['shr_tkfit_2cm_nhits_Y']+CVDF['shr_tkfit_2cm_nhits_U']+CVDF['shr_tkfit_2cm_nhits_V'])
CVDF['shr_tkfit_2cm_dedx_avg'] = (CVDF['shr_tkfit_2cm_nhits_Y']*CVDF['shr_tkfit_2cm_dedx_Y'] + CVDF['shr_tkfit_2cm_nhits_U']*CVDF['shr_tkfit_2cm_dedx_U'] + CVDF['shr_tkfit_2cm_nhits_V']*CVDF['shr_tkfit_2cm_dedx_V'])/CVDF['shr_tkfit_2cm_nhits_tot']
CVDF['shr_tkfit_gap10_nhits_tot'] = (CVDF['shr_tkfit_gap10_nhits_Y']+CVDF['shr_tkfit_gap10_nhits_U']+CVDF['shr_tkfit_gap10_nhits_V'])
CVDF['shr_tkfit_gap10_dedx_avg'] = (CVDF['shr_tkfit_gap10_nhits_Y']*CVDF['shr_tkfit_gap10_dedx_Y'] + CVDF['shr_tkfit_gap10_nhits_U']*CVDF['shr_tkfit_gap10_dedx_U'] + CVDF['shr_tkfit_gap10_nhits_V']*CVDF['shr_tkfit_gap10_dedx_V'])/CVDF['shr_tkfit_gap10_nhits_tot']

CVDF["is_signal"] = CVDF["category"] == 11

INTERCEPT = 0.0
SLOPE = 0.83

CVDF["reco_e"] = (CVDF["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + CVDF["trk_energy_tot"]

#'''

QUERY = "nslice == 1"


CVDF = CVDF.query(QUERY)

print ('there are %i CV events'%(CVDF.shape[0]))

DETSYS_SAMPLE_V = []

for i,N in enumerate(DETVAR_N_V):
    
    #if (i >= 6):
    #    continue
    
    VAR = uproot.open(ls.ntuple_path+DETVAR_S_V[i])[fold][tree]
    VARDF = VAR.pandas.df(variables, flatten=False)
    
    VARDF['identifier'] = VARDF['run']*100000 + VARDF['evt']
    
    VARDF.loc[ VARDF['weightSplineTimesTune'] <= 0, 'weightSplineTimesTune' ] = 1.
    VARDF.loc[ VARDF['weightSplineTimesTune'] == np.inf, 'weightSplineTimesTune' ] = 1.
    VARDF.loc[ VARDF['weightSplineTimesTune'] > 100, 'weightSplineTimesTune' ] = 1.
    VARDF.loc[ np.isnan(VARDF['weightSplineTimesTune']) == True, 'weightSplineTimesTune' ] = 1.

    #'''
    trk_llr_pid_v = VAR.array('trk_llr_pid_score_v')
    trk_energy_proton_v = VAR.array('trk_energy_proton_v')
    trk_calo_energy_y_v = VAR.array('trk_calo_energy_y_v')
    trk_id = VAR.array('trk_id')-1 # I think we need this -1 to get the right result
    trk_llr_pid_v_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_llr_pid_v,trk_id)])
    trk_energy_proton_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_energy_proton_v,trk_id)])
    trk_calo_energy_y_sel = awkward.fromiter([pidv[tid] if tid<len(pidv) else 9999. for pidv,tid in zip(trk_calo_energy_y_v,trk_id)])
    VARDF['trkpid'] = trk_llr_pid_v_sel
    VARDF['protonenergy'] = trk_energy_proton_sel
    VARDF['trackcaloenergy'] = trk_calo_energy_y_sel
    VARDF['subcluster'] = VARDF['shrsubclusters0'] + VARDF['shrsubclusters1'] + VARDF['shrsubclusters2']
    VARDF['trkfit'] = VARDF['shr_tkfit_npointsvalid'] / VARDF['shr_tkfit_npoints']
    VARDF['anglediff_Y'] = np.abs(VARDF['secondshower_Y_dir']-VARDF['shrclusdir2'])

    VARDF['shr_tkfit_2cm_nhits_tot'] = (VARDF['shr_tkfit_2cm_nhits_Y']+VARDF['shr_tkfit_2cm_nhits_U']+VARDF['shr_tkfit_2cm_nhits_V'])
    VARDF['shr_tkfit_2cm_dedx_avg'] = (VARDF['shr_tkfit_2cm_nhits_Y']*VARDF['shr_tkfit_2cm_dedx_Y'] + VARDF['shr_tkfit_2cm_nhits_U']*VARDF['shr_tkfit_2cm_dedx_U'] + VARDF['shr_tkfit_2cm_nhits_V']*VARDF['shr_tkfit_2cm_dedx_V'])/VARDF['shr_tkfit_2cm_nhits_tot']
    VARDF['shr_tkfit_gap10_nhits_tot'] = (VARDF['shr_tkfit_gap10_nhits_Y']+VARDF['shr_tkfit_gap10_nhits_U']+VARDF['shr_tkfit_gap10_nhits_V'])
    VARDF['shr_tkfit_gap10_dedx_avg'] = (VARDF['shr_tkfit_gap10_nhits_Y']*VARDF['shr_tkfit_gap10_dedx_Y'] + VARDF['shr_tkfit_gap10_nhits_U']*VARDF['shr_tkfit_gap10_dedx_U'] + VARDF['shr_tkfit_gap10_nhits_V']*VARDF['shr_tkfit_gap10_dedx_V'])/VARDF['shr_tkfit_gap10_nhits_tot']

    VARDF["reco_e"] = (VARDF["shr_energy_tot_cali"] + INTERCEPT) / SLOPE + VARDF["trk_energy_tot"]
    
    VARDF["is_signal"] = VARDF["category"] == 11 
    #'''
    
    VARDF = VARDF.query(QUERY)
    
    INT = pd.merge(CVDF, VARDF, how='inner', on=['identifier'],suffixes=('_CV', '_VAR'))

    print ('intersection for %15s variation has %i events'%(N,INT.shape[0]))
    
    DETSYS_SAMPLE_V.append(INT)
    
    #break

In [ ]:
def COVARIANCE(n_cv,n_var):
    cov = np.empty([len(n_cv), len(n_cv)])
    cov.fill(0)

    for i in range(len(n_cv)):
        for j in range(len(n_cv)):
            cov[i][j] += (n_var[i] - n_cv[i])*(n_var[j] - n_cv[j])

    frac_cov = np.empty([len(n_cv), len(n_cv)])
    corr = np.empty([len(n_cv), len(n_cv)])

    for i in range(len(n_cv)):
        for j in range(len(n_cv)):
            frac_cov[i][j] =  cov[i][j] / (n_cv[i] * n_cv[j])
            corr[i][j] = cov[i][j] / np.sqrt(cov[i][i] * cov[j][j])
    return cov,frac_cov,corr

In [ ]:
import matplotlib.pyplot as plt

from matplotlib.colors import LogNorm

In [ ]:
VARIABLE = "reco_nu_vtx_sce_x"
TITLE = 'Reconstructed Neutrino Vertex X [cm]'
BINS = np.linspace(0,260,40)

for i,df in enumerate(DETSYS_SAMPLE_V):
    
    SUBQUERY = ""
    
    dfsub = df#.query(TCUTQ)
    
    #if (i!=1):
    #    continue
    
    if (i>=6):
        continue
    
    #print (df)
    idx = i#+6
    
    fig = plt.figure(figsize=(7,5))
    n_cv, bins, p = plt.hist(dfsub['%s_CV'%VARIABLE] ,bins=BINS,histtype='step',lw=2,color='k',\
             label='CV',weights=dfsub['weightSplineTimesTune_CV'])
    n_var, bins, p = plt.hist(dfsub['%s_VAR'%VARIABLE],bins=BINS,histtype='step',lw=2,color='r',\
             label='var : %s'%DETVAR_N_V[idx],weights=dfsub['weightSplineTimesTune_VAR'])
    
    bc = 0.5*(bins[1:]+bins[:-1])
    
    cov,frac_cov,corr = COVARIANCE(n_cv,n_var)
    
    error = np.sqrt(np.diag(frac_cov))
    
    plt.bar(bc,height=2*(error)*n_cv,bottom=n_cv-(error)*n_cv,width=bc[1]-bc[0] ,\
            edgecolor='gray',color='None',lw=2)
    
    plt.xlabel(TITLE)
    plt.ylabel('Num. Entries')
    plt.ylim(0,plt.gca().get_ylim()[1]*1.5)
    plt.legend(fontsize=16,loc=1)
    plt.show()
    fig.savefig(ls.main_path+"detsys/plots/"+VARIABLE+"_"+date_time+"_%s"%DETVAR_N_V[idx]+".pdf")
    
    fig = plt.figure(figsize=(6,6))
    plt.hist2d(dfsub['%s_CV'%VARIABLE],dfsub['%s_VAR'%VARIABLE],bins=(BINS,BINS))#,norm=LogNorm())
    plt.xlabel('%s [CV]'%TITLE)
    plt.ylabel('%s [VAR]'%TITLE)
    plt.title('variation : %s'%DETVAR_N_V[idx],fontsize=16)
    plt.tight_layout()
    plt.plot(BINS,BINS,'r--',lw=2)
    fig.savefig(ls.main_path+"detsys/plots/"+VARIABLE+"_corr_"+date_time+"_%s"%DETVAR_N_V[idx]+".pdf")
    #if (i == 0):

    plt.show()
    
    #continue

            
    fig, ax = plt.subplots(1, 1,figsize=(8,6))
    pos = ax.imshow(frac_cov, origin='lower', cmap='viridis',vmin=-0.1,vmax=0.1)
    ax.set_title("Fractional covariance matrix : %s"%DETVAR_N_V[idx])
    ax.set_ylabel("Bin number")
    ax.set_xlabel("Bin number")
    # Add the text
    # Limits for the extent
    x_start = 0
    x_end = len(n_cv)#-1
    y_start = 0
    y_end = len(n_cv)#-1
    size = len(n_cv)#-1
    
    '''
    jump_x = (x_end - x_start) / (2.0 * size)
    jump_y = (y_end - y_start) / (2.0 * size)
    x_positions = np.linspace(start=x_start, stop=x_end, num=size, endpoint=False)
    y_positions = np.linspace(start=y_start, stop=y_end, num=size, endpoint=False)

    for x_index, x in enumerate(x_positions):
        #for x_index, x in enumerate(x_positions):
        ERR = frac_cov[x_index, x_index]
        label = "%.01f"%(100.*np.sqrt(ERR))
        text_x = x #+ jump_x
        text_y = x #+ jump_y
        if (np.abs(ERR) > 0.05):
            ax.text(text_x, text_y, label, color='black', ha='center', va='center',fontsize=8)
        else:
            ax.text(text_x, text_y, label, color='white', ha='center', va='center',fontsize=8)
    '''
    fig.colorbar(pos)
    

    
    fig, ax = plt.subplots(1, 1)
    ax.set_title("Correlation matrix")
    pos = ax.imshow(corr, origin='lower')
    ax.set_ylabel("Bin number")
    ax.set_xlabel("Bin number")
    fig.colorbar(pos)
